<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Advanced Compilation Options with H-Series

* [H-Series Hardware Compilation](#H-Series-Hardware-Compilation)
  * [Comnp.np.piling to H-Series Hardware Native Gates](#Comnp.piling-to-H-Series-Hardware-Native-Gates)
  * [The General SU(4) Entangler Gate](#The-General-SU(4)-Entangler-Gate)
  * [Controlling H-Series Hardware Compiler Optimizations](#Controlling-H-Series-Hardware-Compiler-Optimizations)
    * [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
    * [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)
* [Examples](#Examples)
    * [1. Benchmarking Circuit in non-native gate set](#1.-Benchmarking-Circuit-in-non-native-gate-set)
    * [2. QFT Circuits in native gate set](#2.-QFT-Circuits-in-native-gate-set)
    * [3. Circuit using $SU(4)$](#Circuit-using-$SU(4)$)

## H-Series Hardware Compilation

Native gates are gates on a quantum computer that the hardware physically executes. Different quantum computers may have different gates that are physically executed on the hardware. Writing a gate in a quantum circuit doesn't guarantee its physical execution on the device. For instance, on H-Series quantum computers, a Hadamard gate written in the circuit is not the actual gate executed. When users submit circuits using a Hadamard gate, the gate is translated into a $U1q$ gate followed by a $Rz$ gate, which the ion trap device physically executes. See the *System Model H1 Product Data Sheet* on the [System Model H1](https://www.quantinuum.com/hardware/h1) page or the *System Model H2 Product Data Sheet* on the [System Model H2](https://www.quantinuum.com/hardware/h2) page for a listing of the H-Series hardware native gates. 

The H-Series hardware compiler handles the translation from circuits users submit to the native gates run on hardware. In the H-Series Quantum Charge-Coupled Device (QCCD) architecture, the hardware compilation includes the assignment of which physical qubit corresponds to which qubit in a circuit as well as how qubits will be transported around the device. Since transport, as well as gating, incurs a small amount of error with each operation, the H-Series compiler aims to minimize the number of gates that need to be executed. 

<div style="text-align: center;">
         <img src="figureshseries-compilation-stac6.png" width="400" />
</div>

### Compiling to H-Series Hardware Native Gates

On the Quantinuum H-Series devices, there are different native two-qubit gates available. The default native two-qubit gates include a fully entangling two-qubit gate, $ZZ()$, and an arbitrary angle ZZ gate, $Rzz(\theta)$. Another native gate available on the hardware is the General $SU(4)$ Entangler gate.

By default, the hardware compiler compiles to the $ZZ()$ or $Rzz(\theta)$ gate. Currently, only one native gate can be specified at a time. This ensures everything aligns in the global operations of the circuit. 

If users would like to use the General $SU(4)$ Entangler gate and not have the circuit rebased to $ZZ()$ or $Rzz(\theta)$ by the hardware compiler, they need to specify the $SU(4)$ gate using the `nativetq` option. The `nativetq` option is available to override the hardware stack's default two-qubit gate and use the supplied gate instead.

* `nativetq`: override the stack's default native two-qubit gate and use the supplied gate as the gate instead
  * `ZZ`: compile circuit to the $ZZ$ gate
  * `RZZ`: compile circuit to the $Rzz(\theta)$ gate, known as the `Optype.ZZPhase` gate within `pytket`.
  * `Rxxyyzz`: compile circuit to the $SU(4)$ gate, known as `Optype.TK2` within `pytket`.

### The General $SU(4)$ Entangler Gate

The General $SU(4)$ Entangler gate, or `Rxxyyzz`, is available in TKET as [`OpType.TK2`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.TK2). This gate is a combination of `OpType.XXPhase`, `OpType.YYPhase` and `OpType.ZZPhase`, and requires three angles as input, $\alpha$, $\beta$ and $\gamma$. The definition of the gate is provided below:

$$\begin{equation} \textrm{TK2}(\alpha, \beta, \gamma) = e^{-\frac{1}{2} i \pi \alpha (\hat{X} \bigotimes \hat{X})} \quad e^{-\frac{1}{2} i \pi \beta (\hat{Y} \bigotimes \hat{Y})} \quad e^{-\frac{1}{2} \pi \gamma (\hat{Z} \bigotimes \hat{Z})} = e^{-\frac{1}{2} i \alpha (\hat{X} \bigotimes \hat{X}) + \beta (\hat{Y} \bigotimes \hat{Y}) + \gamma (\hat{Z} \bigotimes \hat{Z})} \end{equation} $$

This gate can be used as follows within TKET.

In [1]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from sympy import Symbol

symbols = [Symbol("a"), Symbol("b"), Symbol("c")]
circuit = Circuit(2)
circuit.TK2(*symbols, *circuit.qubits)
render_circuit_jupyter(circuit)

This circuit can be converted to a QASM string using the [`circuit_to_qasm_str`](https://tket.quantinuum.com/api-docs/qasm.html#pytket.qasm.circuit_to_qasm_str) function and by specifying the Quantinuum header `hqslib1`.

In [2]:
from pytket.qasm.qasm import circuit_to_qasm_str

print(circuit_to_qasm_str(circuit, header="hqslib1"))

OPENQASM 2.0;
include "hqslib1.inc";

qreg q[2];
Rxxyyzz((a)*pi,(b)*pi,(c)*pi) q[0],q[1];



### Controlling H-Series Hardware Compiler Optimizations

Users have the option of submitting circuits using whichever quantum gate set they desire. Users do not need to think about which physical gates will be executed or how physical qubits will move around the device since the hardware compiler manages this. In certain cases, however, users may want to know that the circuit they submit is going to be run on the device exactly as they write it. For example, when running benchmarking circuits users may want circuits to be executed exactly as specified in the circuit even if its not the most optimal in total number of two-qubit gates. 

Within the Quantinuum stack, the ability to control levels of TKET optimizations and control over what is executed on the hardware is provided between 4 different job submission parameters in the API.

There are two ways to think about using these options:
1. [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
2. [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)

#### Circuits written in any gate set

Users are free to submit circuits written with any gate set, not just the native gate set of the hardware. In this case, the options for control over what optimizations are applied are given at the TKET level. TKET will rebase the circuit to the native gate set it believes is most optimal and the hardware compiler will handle further optimizations of gate combinations as it applies to transport and ion assignment. **We recommend this for the majority of use cases.**

* `tket-opt-level`: the `tket` optimization level to apply (default: `2`), with `tket` optimizations turned on, the hardware compiler will provide further gate combination logic as makes sense for ions and transport
  * `2`: powerful optimizations, can use approximate methods, compilation can be expensive
  * `1`: basic optimization, compiles quickly
  * `0`: rebase the circuit with `tket`
  * `null`: rebase the circuit without `tket`, using the hardware compiler only

#### Circuits written in the hardware's native gate set

For circuits that are written only using gates in the hardware's native gate set, various levels of control are provided for what optimizations will be performed in the stack.

Users may still choose to apply `tket` optimizations specified above to their circuit and control this with `tket-opt-level`, even if the circuit is written in the native gate set since further reductions in the number of quantum gates may be found, which will improve results. 

To use the native gates and circuit as the user submits it `tket-opt-level` must be set to `null` when submitting and the following options must be used.

* `no-opt`: turns off all `tket` optimizations *and* all hardware compiler gate combination logic. (default: `False`)
    * If more than 1 native gate is used in the circuit, the circuit will be rebased to 1 native gate, but no further gate combination logic will occur.
    * The job will fail if `no-opt` is set to `True` and the circuit contains non-native gates. 
* `noreduce`: turns off all `tket` optimizations, all hardware compiler gate combination logic, and requires exact 1:1 correspondence of two-qubit gates with gates on the system. This requires the circuit be submitted using one of the native two-qubit gates on the system, otherwise an error will be returned. (default: `False`)

Note that `tket-opt-level` set to `null`, `no-opt` set to `True`, and `noreduce` set to `True` all disable `tket` optimizations, but `no-opt` and `noreduce` also turn off all hardware gate combination logic.

**Create plot of quantum circuit -> tket opt or no-opt or nativetq or noreduce**

## Examples

Now we illustrate the above options with a few examples. First we import the functions we need in `pytket`.   

In [3]:
import numpy as np
import pandas as pd

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.quantinuum import QuantinuumBackend

In [4]:
from pytket.qasm.qasm import circuit_to_qasm_str, circuit_from_qasm, circuit_from_qasm_str

### Define default native two-qubit gate with `QuantinuumBackend`

In [22]:
from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.quantinuum.backends.quantinuum import QuantinuumBackendCompilationConfig

from pytket.circuit import OpType

compilation_config = QuantinuumBackendCompilationConfig(allow_implicit_swaps=True, target_2qb_gate=OpType.TK2)
compilation_config.target_2qb_gate

<OpType.TK2: 41>

In [23]:
quantinuum_backend = QuantinuumBackend(device_name="H1-1E", compilation_config=compilation_config, options={"tket-op-level": None})
quantinuum_backend.login()

### Quantum Volume Circuits with and without the General $SU(4)$ Entangler

Quantum volume is a benchmarking test that was initially proposed by IBM [TODO: insert reference]. It is a test that aims to verify the quality as well as the quantity of qubits on the machine. The test does this by peforming rounds of single and two qubit gates between random pairs of qubits for as many rounds as qubits in the test. For example, for quantum volume of 2^N where N=4, 4 random rounds of gates are performed. If the quantum computer performing the test passes, it verifies that the quantum computer can perform quality computation with equal depth as width of the circuit. The advantage to using quantum volume is that it gives users the confidence that not only do they have the number of qubits to support running their circuit, but the two-qubit gate fidelity is there to support circuits of significant depth as well. Quantum algorithms need not only qubits, but the ability to run many gates consecutively as well. Quantinuum has steadily been increasing the quantum volume of H-Series machines. 

In [24]:
qv_circuit_wSU4 = Circuit(4, name="QV Example with SU(4)")

# Round 1
qv_circuit_wSU4.U3(3.2465066100946998*np.pi, 2.5284408131911524*np.pi, 4.1734440424380335*np.pi, 0)
qv_circuit_wSU4.U3(3.56126038967905*np.pi, 0.5417259593027202*np.pi, 0.5889430256705375*np.pi, 1)
qv_circuit_wSU4.U3(3.4457004625001506*np.pi, 2.238803660106308*np.pi, 2.666707582855283*np.pi, 2)
qv_circuit_wSU4.U3(3.6550299908952426*np.pi, -0.09542555037191236*np.pi, 2.2959051047685524*np.pi, 3)

qv_circuit_wSU4.TK2(0.3747103659393297*np.pi, 0.25045542681642363*np.pi, 0.16594061155215553*np.pi, 0, 2)
qv_circuit_wSU4.TK2(0.4049821668324701*np.pi, 0.3201934298378358*np.pi, 0.14725271101077092*np.pi, 1, 3)

# Round 2
qv_circuit_wSU4.U3(3.9176110504261077*np.pi, 2.9748059513568568*np.pi, 3.5892285844018543*np.pi, 0)
qv_circuit_wSU4.U3(3.057201228666834*np.pi, 2.3577755337975868*np.pi, 3.7508467917045114*np.pi, 1)
qv_circuit_wSU4.U3(3.5721279036823654*np.pi, 0.05730037730060722*np.pi, 3.2791170965521705*np.pi, 2)
qv_circuit_wSU4.U3(3.535724388746909*np.pi, 0.05595930815342476*np.pi, 3.1299299416986477*np.pi, 3)

qv_circuit_wSU4.TK2(0.43955814978837976*np.pi, 0.3294379018024353*np.pi, 0.22483582064207827*np.pi, 0, 3)
qv_circuit_wSU4.TK2(0.30761522377699807*np.pi, 0.24902805179116916*np.pi, 3.9421010293613925*np.pi, 1, 2)

# Round 3
qv_circuit_wSU4.U3(3.024862371165571*np.pi, 2.3296051463212555*np.pi, 4.47465538899325*np.pi, 0)
qv_circuit_wSU4.U3(3.3793899755344796*np.pi, 1.2958915291565876*np.pi, 0.9370112066521646*np.pi, 1)
qv_circuit_wSU4.U3(3.466056716208809*np.pi, 3.4526574409721653*np.pi, 1.2690606129862783*np.pi, 2)
qv_circuit_wSU4.U3(3.3138153252630116*np.pi, 2.996929687445126*np.pi, 3.2634131850838957*np.pi, 3)

qv_circuit_wSU4.TK2(0.48089722439645044*np.pi, 0.21393048476072266*np.pi, 3.92648010233256*np.pi, 0, 2)
qv_circuit_wSU4.TK2(0.33895792181776946*np.pi, 0.2518811716342433*np.pi, 0.13342119970597022*np.pi, 1, 3)

# Round 4
qv_circuit_wSU4.U3(3.787489505597836*np.pi, -0.2026128585756619*np.pi, 1.1655980797683991*np.pi, 0)
qv_circuit_wSU4.U3(1.0261490063596848*np.pi, 0.16193977105724777*np.pi, 1.6325026187234268*np.pi, 1)
qv_circuit_wSU4.U3(3.330882266512557*np.pi, 2.2076389369717675*np.pi, 0.752751605442116*np.pi, 2)
qv_circuit_wSU4.U3(2.5144296195023776*np.pi, 0.04059434060036249*np.pi, 4.306225020434145*np.pi, 3)
qv_circuit_wSU4.measure_all()

[U3(2.1992, 1.94333, 1.11126) q[0]; U3(3.18803, 1.70188, 1.85022) q[1]; U3(2.82499, 1.03341, 0.377709) q[2]; U3(3.48262, 1.70021, 1.2128) q[3]; TK2(1.17719, 0.786829, 0.521318) q[0], q[2]; TK2(1.27229, 1.00592, 0.462608) q[1], q[3]; U3(0.307538, 1.34563, 1.27589) q[0]; U3(1.60448, 1.40717, 1.78363) q[1]; U3(3.22217, 0.180014, 0.30165) q[2]; U3(3.10781, 0.175801, 1.83296) q[3]; TK2(1.38091, 1.03496, 0.706343) q[0], q[3]; TK2(0.966402, 0.782345, 0.384476) q[1], q[2]; U3(1.50289, 1.31867, 0.0575445) q[0]; U3(2.61667, 0.0711633, 0.943708) q[1]; U3(2.88894, 0.846843, 1.98687) q[2]; U3(2.41066, 1.41513, 0.252315) q[3]; TK2(1.51078, 0.672082, 0.335401) q[0], q[2]; TK2(1.06487, 0.791308, 0.419155) q[1], q[3]; U3(3.89875, 1.36347, 1.66183) q[0]; U3(3.22374, 0.508749, 1.12866) q[1]; U3(2.46428, 0.935502, 0.364839) q[2]; U3(3.89931, 0.127531, 1.5284) q[3]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; Measure q[2] --> c[2]; Measure q[3] --> c[3]; ]

In [28]:
compiled_circuit_wsu4 = quantinuum_backend.get_compiled_circuit(qv_circuit_wSU4, optimisation_level=2)
print(f"Number of TK2 (Rxxyyzz) Gates: {compiled_circuit_wsu4.n_2qb_gates()}")
render_circuit_jupyter(compiled_circuit_wsu4)

Number of TK2 (Rxxyyzz) Gates: 18


In [30]:
cost = quantinuum_backend.cost(compiled_circuit_wsu4, n_shots=100, syntax_checker="H1-1SC")
print(cost)

10.2


In [32]:
handle = quantinuum_backend.process_circuit(compiled_circuit_wsu4, n_shots=100)

In [36]:
result = quantinuum_backend.get_result(handle)
print(result.get_distribution())

{(0, 0, 0, 0): 0.08, (0, 0, 0, 1): 0.05, (0, 0, 1, 0): 0.03, (0, 1, 0, 0): 0.06, (0, 1, 0, 1): 0.24, (0, 1, 1, 1): 0.03, (1, 0, 0, 0): 0.03, (1, 0, 0, 1): 0.03, (1, 0, 1, 0): 0.04, (1, 1, 0, 0): 0.04, (1, 1, 0, 1): 0.14, (1, 1, 1, 0): 0.01, (1, 1, 1, 1): 0.22}


In [14]:
quantinuum_backend.set_compilation_config_target_2qb_gate(OpType.ZZPhase)

In [15]:
qv_circuit_no_SU4 = Circuit(4, name="QV Example without SU(4)")

qv_circuit_no_SU4.U3(0.25473320252075615*np.pi,-0.4722839623120936*np.pi,1.1337576181565758*np.pi, 0)
qv_circuit_no_SU4.U3(0.9260248851696505*np.pi,2.6880694641488283*np.pi,3.781045841193389*np.pi, 1)
qv_circuit_no_SU4.U3(0.2654757321735285*np.pi,0.0736141120852053*np.pi,4.116758761767741*np.pi, 2)
qv_circuit_no_SU4.U3(0.4159547432138166*np.pi,0.16111290510915977*np.pi,0.7501741218212356*np.pi, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.CX(1, 3)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,3.4049821668324665*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.3201934298378362*np.pi, 3)
qv_circuit_no_SU4.CX(0, 2)
qv_circuit_no_SU4.CX(1, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,3.3747103659393294*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.2504554268164243*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,0.6472527110107711*np.pi, 3)
qv_circuit_no_SU4.CX(0, 2)
qv_circuit_no_SU4.CX(1, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,0.6659406115521548*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.CX(0, 2)
qv_circuit_no_SU4.U3(0.25535963651944854*np.pi,3.1014277080861374*np.pi,0.8687871783780351*np.pi, 1)
qv_circuit_no_SU4.U3(0.4242946197076395*np.pi,0.2614656370005698*np.pi,3.4838692502872806*np.pi, 3)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.7648074607597222*np.pi,-0.22109305574283394*np.pi,4.270484169822186*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.22451081382324406*np.pi,0.27225730267610504*np.pi,3.927244666700967*np.pi, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.66949436019446*np.pi,2.5791859689113434*np.pi,3.699026291695842*np.pi, 0)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.6909969444247932*np.pi,-0.17170015925367116*np.pi,1.0193666698299448*np.pi, 0)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.8542699271757708*np.pi,3.1239275299676947*np.pi,2.8820137020693153*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5503794877412841*np.pi,1.059003801067796*np.pi,0.9221441436832603*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.CX(0, 3)
qv_circuit_no_SU4.CX(1, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,3.4293302654969113*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,3.4810601554611322*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.3079118604508042*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.2821248812866428*np.pi, 3)
qv_circuit_no_SU4.CX(0, 3)
qv_circuit_no_SU4.CX(1, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,0.7066487742208293*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,0.568720952507614*np.pi, 3)
qv_circuit_no_SU4.CX(0, 3)
qv_circuit_no_SU4.CX(1, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.7042706963515156*np.pi,0.8569341596745985*np.pi,4.382539932255121*np.pi, 0)
qv_circuit_no_SU4.U3(0.5601447710772369*np.pi,1.09199384065995*np.pi,0.7308152830574234*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.2266612071559701*np.pi,0.16782619554269707*np.pi,1.021229946537722*np.pi, 0)
qv_circuit_no_SU4.U3(0.2762246138123449*np.pi,0.4576836920669646*np.pi,4.104878292146786*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,4.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.3992168256084478*np.pi,3.32099255409016*np.pi,4.316031829539449*np.pi, 2)
qv_circuit_no_SU4.U3(0.17449638263602396*np.pi,0.6388780639818601*np.pi,0.7170079306262069*np.pi, 3)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.04056918324275033*np.pi,2.4975811387857263*np.pi,1.1688582372774654*np.pi, 2)
qv_circuit_no_SU4.U3(0.7425014918208815*np.pi,2.5513295057684955*np.pi,3.798716571514106*np.pi, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,4.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,4.0*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.CX(0, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,4.0*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,3.489001455095616*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.2533359797863637*np.pi, 3)
qv_circuit_no_SU4.CX(0, 3)
qv_circuit_no_SU4.CX(1, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,3.4878835207861334*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.4143578707787503*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,0.5810281952595922*np.pi, 3)
qv_circuit_no_SU4.CX(0, 3)
qv_circuit_no_SU4.CX(1, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,4.129235226639673*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.858475273889848*np.pi,3.344516428913573*np.pi,3.0600916412099446*np.pi, 0)
qv_circuit_no_SU4.CX(1, 2)
qv_circuit_no_SU4.U3(0.8245657987254952*np.pi,0.12473351582941017*np.pi,0.8623599356030089*np.pi, 3)
qv_circuit_no_SU4.U3(0.4946111147607045*np.pi,-0.040647131861198005*np.pi,3.594081508778993*np.pi, 0)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.8280822479969498*np.pi,3.3567211044985177*np.pi,3.7860353910345665*np.pi, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,4.0*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,4.0*np.pi, 2)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5554762677015984*np.pi,3.2859255637424867*np.pi,4.289308300879274*np.pi, 1)
qv_circuit_no_SU4.U3(0.6584101141439711*np.pi,2.779577244162456*np.pi,3.721888657269012*np.pi, 2)
qv_circuit_no_SU4.U3(0.8153952155638982*np.pi,-0.008997488527685837*np.pi,3.9172737166210907*np.pi, 1)
qv_circuit_no_SU4.U3(0.5055323943045988*np.pi,0.20235580219106175*np.pi,3.602501861285357*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.CX(0, 2)
qv_circuit_no_SU4.U3(1.5*np.pi,0.0*np.pi,1.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,3.480897224396455*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.CX(1, 3)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.213930484760723*np.pi, 2)
qv_circuit_no_SU4.CX(0, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,3.338957921817773*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(1.0*np.pi,0.0*np.pi,1.2518811716342444*np.pi, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.CX(1, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,4.426480102332563*np.pi, 2)
qv_circuit_no_SU4.CX(0, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,0.6334211997059697*np.pi, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.CX(1, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.026149006359698555*np.pi,0.16193977105725021*np.pi,3.367497381276574*np.pi, 1)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 3)
# qv_circuit_no_SU4.Measure(1, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(1.0*np.pi,-0.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.U3(0.40042121316676804*np.pi,3.4319547249435396*np.pi,0.6909747338081911*np.pi, 0)
qv_circuit_no_SU4.U3(0.0*np.pi,-0.5*np.pi,1.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5144296195023755*np.pi,0.04059434060036193*np.pi,1.306225020434144*np.pi, 3)
# qv_circuit_no_SU4.Measure(0, 0)
# qv_circuit_no_SU4.Measure(3, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,-0.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.7065628237937605*np.pi,-0.1452950236479406*np.pi,4.219378673097818*np.pi, 2)
# qv_circuit_no_SU4.Measure(2, 2)
qv_circuit_no_SU4.measure_all()

[U3(0.800268, 0.516276, 1.5618) q[0]; U3(2.90919, 0.444819, 1.87851) q[1]; U3(0.834017, 0.231266, 0.933179) q[2]; U3(1.30676, 0.506151, 0.356742) q[3]; U3(2.99557, 0.429204, 1.5708) q[0]; U3(2.99557, 0.429204, 1.5708) q[1]; U3(2.99557, 0.429204, 1.5708) q[2]; U3(2.99557, 0.429204, 1.5708) q[3]; U3(0, 0.429204, 0.712389) q[0]; U3(0.712389, 0, 0.712389) q[1]; U3(1.5708, 0.429204, 1.5708) q[2]; U3(1.5708, 0.429204, 1.5708) q[3]; U3(3.14159, 0.429204, 1.5708) q[0]; CX q[1], q[3]; U3(0.712389, 0, 0.712389) q[0]; U3(2.99557, 0.697067, 1.5708) q[1]; U3(3.14159, 0, 0.14751) q[3]; CX q[0], q[2]; CX q[1], q[3]; U3(2.99557, 0.601965, 1.5708) q[0]; U3(1.5708, 0, 1.5708) q[1]; U3(3.14159, 0, 1.92842) q[2]; U3(0, 0.429204, 0.0334044) q[3]; CX q[0], q[2]; CX q[1], q[3]; U3(1.5708, 0, 1.5708) q[0]; U3(1.5708, 0.429204, 1.5708) q[1]; U3(0, 0.429204, 0.0921141) q[2]; U3(1.5708, 0.429204, 1.5708) q[3]; CX q[0], q[2]; U3(0.802236, 1.74342, 0.729375) q[1]; U3(1.33296, 0.821419, 0.944898) q[3]; U3(3.14159, 

In [16]:
compiled_circuit_no_su4 = quantinuum_backend.get_compiled_circuit(qv_circuit_no_SU4, optimisation_level=2)
print(f"Number of ZZPhase Gates: {compiled_circuit_no_su4.n_2qb_gates()}")
render_circuit_jupyter(compiled_circuit_no_su4)

Number of ZZPhase Gates: 18


<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>